In [1]:
import numpy as np
import scipy.linalg as la
import scipy.sparse as sparse
import gradientDescent as gd

In [2]:
import imp

In [23]:
imp.reload(gd)

<module 'gradientDescent' from '/Users/andreamarin/Desktop/ITAM/9semestre/Analisis_Aplicado/Laboratorios/gradientDescent.py'>

Interpolación cuadrática (quiero que I(x) = phi(x))

I(x) = a + bx + c(x^2)

    I(0) = phi(0) --> a = phi(0)
    I'(0) = phi'(0) --> b = phi'(0)
    I(x0) = phi(x0) --> c*x0^2 = phi(x0) -a - b*x0

In [45]:
def max_descent_cuad(f, x0, tol=1e-5, max_iter=100, Df = None, Q = None):
    '''
    Máximo descenso con paso óptimo para funciones cuadráticas
    
    params:
        f - funcion a minimizar
        x0 - punto inicial
        tol - tolerancia
        max_iter - cota superior p/iteraciones
        Df (opcional) - gradiente de f
        Q (opcional) - hessiano de f
        
    returns:
        xf - aproximación de x*
        iter - número de iteraciones usadas
    '''
    
    xf = x0
    n = len(x0)
    iterations = 0
    c_1 = 0.1
    
    grad_f = gd.grad(f, xf) if Df is None else Df(xf)
    hess_f = gd.hess(f, xf) if Q is None else Q(xf)
    
    
    while iterations < max_iter and np.linalg.norm(grad_f, np.inf) > tol:
        
        #Find descent direction
        d = - grad_f / np.linalg.norm(grad_f)
        
        #Direction coef.
        d_t = np.transpose(d)
        
        alpha = np.dot(-d_t,grad_f)/np.dot(d_t,np.dot(hess_f,d))
        alpha = 0.99*alpha
        
        # assert function
        slope = c_1*np.dot(grad_f, d)
        assert f(xf + alpha*d) <= f(xf) + alpha*slope, 'No cumple condicion'
        
        #Next iteration
        xf = xf + alpha*d
        iterations += 1
        
        grad_f = gd.grad(f, xf) if Df is None else Df(xf)
    
    return xf, iterations

In [41]:
def max_descent_opt(f, x0, tol = 1e-5, max_iter = 100):
    '''
    Desciende en la dirección de descenso más grande (para funciones no cuadráticas)
    '''
    
    xf = x0
    n = len(x0)
    iterations = 0
    
    c_1 = 0.1
    c_2 = 0.9
    
    grad_f = gd.grad(f, xf)
    
    
    while iterations < max_iter and np.linalg.norm(grad_f, np.inf) > tol:
        #Find descent direction
        d = - grad_f / np.linalg.norm(grad_f)
        
        # Interpolation parameters
        a = f(xf)
        b = np.dot(grad_f,d)
        alpha0 = gd.bissec(grad_f, d, c_1, f, xf)
        
        #Direction coef.
        alpha = (-0.5*b*(alpha0**2))/(f(xf+alpha0*d) - a - b*alpha0)
        alpha = 0.99*alpha
        
        # assert function (W1)
        #assert f(xf + alpha*d) <= f(xf) + alpha*c_2*b, 'No cumple condicion W2'
        
        #Next iteration
        xf = xf + alpha*d
        iterations += 1
        
        grad_f = gd.grad(f, xf)
        
        # assert function (W2)
        #assert np.dot(grad_f,d) > c2*b, 'No cumple condicion 2'
    
    return xf, iterations

### Pruebas

In [14]:
max_iter = 2000

Pascal function

In [28]:
A = la.pascal(4)
b = -np.ones(4)

x0 = np.array([4,4,4,4])

f = lambda x : 1/2 * np.dot(x, np.dot(A, x)) + np.dot(b, x) + 1

In [29]:
max_descent_cuad(f, x0, max_iter = max_iter)

(array([ 9.99914450e-01,  1.97823741e-04, -1.63813656e-04,  4.71048635e-05]),
 86)

In [16]:
Df = lambda x: np.dot(A, x) + b
Q = lambda x: A

max_descent_cuad(f, x0, max_iter = max_iter, Df = Df, Q = Q)

(array([ 9.99915062e-01,  1.96183061e-04, -1.62695707e-04,  4.63103130e-05]),
 87)

In [37]:
max_descent_opt(f, x0, max_iter = max_iter)

(array([ 9.99902989e-01,  2.27165057e-04, -1.86929806e-04,  5.27159943e-05]),
 1549)

Rosenbrock

In [46]:
rosenbrock = lambda x : 100*(x[0]**2 - x[1])**2 + (x[0]-1)**2
x0_r = np.array([2, 3])

max_descent_cuad(rosenbrock, x0_r, max_iter = max_iter)

AssertionError: No cumple condicion

In [47]:
max_descent_opt(rosenbrock, x0_r, max_iter = max_iter)

(array([1.00000822, 1.00001648]), 471)